#### **Projekt1**

Pre celkovo najlepšie natrénovanú CNN, zobrazte vybrané filtre v 1. a 2. konvolučnej vrstve, zobrazte výslednú mapu príznakov pre vybraný vstupný obraz. 


In [ ]:
import pickle
import numpy as np
import torch
#from pretty_confusion_matrix import pp_matrix_from_data
from models_architecture import *
from matplotlib import pyplot as plt
from torchvision import utils
from dataset_architecture import get_architectural_dataset
from torchvision import transforms

Best network

In [ ]:
#modify accordingly
model = resnet18_architecture()
name = model.__class__.__name__
run='1'

path = 'trained_models'


model.load_state_dict(torch.load(f"{path}/{name}/{run}/model.pt"))

In [ ]:

def visTensor(tensor, ch=0, allkernels=False, nrow=8, padding=1,title="Convolution layer filters"):
    n, c, w, h = tensor.shape

    if allkernels:
        tensor = tensor.view(n * c, -1, w, h)
    elif c != 3:
        tensor = tensor[:, ch, :, :].unsqueeze(dim=1)

    rows = np.min((tensor.shape[0] // nrow + 1, 64))
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    plt.figure(figsize=(nrow, rows))
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.title(title)
    # plt.show()

Filters of convolutional layers 1 and 2

In [ ]:
print(model)

print("Displaying layer 1")
filter_1 = model.conv1.weight.data.clone().cpu()
%matplotlib qt
visTensor(filter_1, ch=0, allkernels=False, title="Convolution layer 1 filters")

print("Displaying layer 2")
filter_2 = model.layer1[0].conv1.weight.data.clone().cpu()
visTensor(filter_2, ch=0, allkernels=False, title="Convolution layer 2 filters")


#filter = cnn2.conv1[0].weight.data.clone().cpu()
#visTensor(filter, ch=0, allkernels=False)

Displaying feature maps

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize(256, transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.ConvertImageDtype(dtype=torch.float32),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

trainloader, _ = get_architectural_dataset(
    root_path="architectural-styles-dataset/",
    #root_path="C:\\Repositories\\hns\\Dataset_Architektura\\architectural-styles-dataset", 
    transform=transform,
    batch_sz=28,
    test=0.1,
)

In [ ]:
#extracting one sample
for data, target in trainloader:
    single_data = data[0]
    single_target = target[0]
    break

In [ ]:
#showing feature map
image = single_data.permute(1, 2, 0).numpy()
#print (image)
image = (image + 1) / 2.0
image = np.clip(image, 0.0, 1.0)
plt.imshow(image)

model.eval()

with torch.no_grad():
    output = model(single_data.unsqueeze(0))
    conv_output = model.conv1(single_data.unsqueeze(0))

_, predicted = torch.max(output.data, 1)

print(f"Predicted label: {predicted.cpu().numpy()} | True label: {single_target}")


In [ ]:
# Squeeze the batch dimension
conv_output = conv_output.squeeze(0)

# Number of rows/columns for visualization grid
grid_size = 8  

fig, axs = plt.subplots(grid_size, grid_size, figsize=(15,15))

for i in range(grid_size):
    for j in range(grid_size):
        # Extract the feature map
        fmap = conv_output[i*grid_size + j].cpu().numpy()

        # Display the feature map
        axs[i, j].imshow(fmap, cmap='gray')
        axs[i, j].axis('off')
        
fig.suptitle('Feature maps', fontsize=20)
plt.show()